# Block Aligner Benchmark Analysis and Visualizations

This notebook contains code for collecting, cleaning, and analyzing data produced by block aligner's experiments.

To run this, you will need to install all the libraries imported below, along with [altair-saver](https://github.com/altair-viz/altair_saver) and [altair-data-server](https://github.com/altair-viz/altair_data_server), which has some extra dependencies for PDF saving.

Run each cell one by one to reproduce the experiments. This may take a while. For accurate benchmarking, it is recommended to run the entire notebook in the command line with `nbconvert`.

In [1]:
import altair as alt
from altair_saver import save
from altair import datum
import pandas as pd
from io import StringIO

alt.data_transformers.enable("data_server")

DataTransformerRegistry.enable('data_server')

In [2]:
def csv_to_pandas(csv, d = "\\s*,\\s*", t = None):
    s = StringIO("\n".join(csv))
    data = pd.read_csv(s, sep = d, thousands = t, comment = "#", engine = "python")
    return data

## Prefix Scan Benchmark

In [3]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- prefix_scan | grep 'bench:' | awk '{print $2"\t"$5}'
output.insert(0, "algorithm\ttime")
output

['algorithm\ttime', 'bench_naive_prefix_scan\t25', 'bench_opt_prefix_scan\t17']

In [4]:
data = csv_to_pandas(output, d = "\t", t = ",")
data

,algorithm,time
0,bench_naive_prefix_scan,25
1,bench_opt_prefix_scan,17


In [5]:
data["algorithm"] = data["algorithm"].map({
    "bench_naive_prefix_scan": "naive",
    "bench_opt_prefix_scan": "ours"
})
data

,algorithm,time
0,naive,25
1,ours,17


Prefix Scan Benchmark (AVX2)

In [6]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("time", axis = alt.Axis(title = "time (ns)")),
    y = "algorithm",
    color = alt.Color("algorithm", legend = None)
).properties(
    width = 150
)
save(c, "prefix_scan_bench.pdf")
c

alt.Chart(...)

## Random Data Benchmark

In [7]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- bench_ | grep 'bench:' | grep -v 'prefix_scan' | awk '{print $2"\t"$5}'
output

['bench_parasailors_aa_1000_10000\t46,116,339',
 'bench_parasailors_aa_100_1000\t575,601',
 'bench_parasailors_aa_10_100\t20,196',
 'bench_rustbio_aa_100_1000\t14,649,204',
 'bench_rustbio_aa_10_100\t155,004',
 'bench_scan_aa_1000_10000\t231,733',
 'bench_scan_aa_1000_10000_insert\t4,803,359',
 'bench_scan_aa_1000_10000_small\t221,973',
 'bench_scan_aa_1000_10000_trace\t1,845,369',
 'bench_scan_aa_100_1000\t24,075',
 'bench_scan_aa_100_1000_insert\t51,366',
 'bench_scan_aa_100_1000_small\t22,868',
 'bench_scan_aa_100_1000_trace\t506,968',
 'bench_scan_aa_10_100\t3,888',
 'bench_scan_aa_10_100_insert\t3,985',
 'bench_scan_aa_10_100_small\t3,412',
 'bench_scan_aa_10_100_trace\t355,171',
 'bench_scan_nuc_1000_10000\t226,318',
 'bench_scan_nuc_100_1000\t23,150',
 'bench_triple_accel_1000_10000\t7,809,720',
 'bench_triple_accel_100_1000\t23,337']

In [8]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["parasailors_aa", "rustbio_aa", "scan_aa", "scan_nuc", "triple_accel"]
new_names = ["parasailors\tprotein", "rust bio\tprotein", "ours\tprotein", "ours\tnucleotide", "triple accel\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'parasailors\tprotein\t1000\t10000\tdefault\t46,116,339',
 'parasailors\tprotein\t100\t1000\tdefault\t575,601',
 'parasailors\tprotein\t10\t100\tdefault\t20,196',
 'rust bio\tprotein\t100\t1000\tdefault\t14,649,204',
 'rust bio\tprotein\t10\t100\tdefault\t155,004',
 'ours\tprotein\t1000\t10000\tdefault\t231,733',
 'ours\tprotein\t1000\t10000\tinsert\t4,803,359',
 'ours\tprotein\t1000\t10000\tsmall\t221,973',
 'ours\tprotein\t1000\t10000\ttrace\t1,845,369',
 'ours\tprotein\t100\t1000\tdefault\t24,075',
 'ours\tprotein\t100\t1000\tinsert\t51,366',
 'ours\tprotein\t100\t1000\tsmall\t22,868',
 'ours\tprotein\t100\t1000\ttrace\t506,968',
 'ours\tprotein\t10\t100\tdefault\t3,888',
 'ours\tprotein\t10\t100\tinsert\t3,985',
 'ours\tprotein\t10\t100\tsmall\t3,412',
 'ours\tprotein\t10\t100\ttrace\t355,171',
 'ours\tnucleotide\t1000\t10000\tdefault\t226,318',
 'ours\tnucleotide\t100\t1000\tdefault\t23,150',
 'triple accel\tnucleotide\t1000\t100

In [9]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,parasailors,protein,1000,10000,default,46116339
1,parasailors,protein,100,1000,default,575601
2,parasailors,protein,10,100,default,20196
3,rust bio,protein,100,1000,default,14649204
4,rust bio,protein,10,100,default,155004
5,ours,protein,1000,10000,default,231733
6,ours,protein,1000,10000,insert,4803359
7,ours,protein,1000,10000,small,221973
8,ours,protein,1000,10000,trace,1845369
9,ours,protein,100,1000,default,24075


In [10]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (AVX2)

In [11]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench.pdf")
c

alt.Chart(...)

Random DNA Sequences Benchmark (AVX2)

In [12]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000])),
    shape = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000]))
).transform_filter(
    datum.alphabet == "nucleotide"
).properties(
    width = 200,
    height = 50
)
save(c, "random_dna_bench.pdf")
c

alt.Chart(...)

## Uniclust 30 Data Benchmark

In [13]:
output = !cd .. && cargo run --example uc_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, size, time',
 'ours (no trace), uc30, 32-32, 0.056956084',
 'ours (no trace), uc30 0.95, 32-32, 0.061715117',
 'ours (no trace), uc30, 32-256, 0.088622874',
 'ours (no trace), uc30 0.95, 32-256, 0.075699876',
 'ours (no trace), uc30, 256-256, 0.199376034',
 'ours (no trace), uc30 0.95, 256-256, 0.22471913',
 'ours (trace), uc30, 32-256, 0.169768454',
 'ours (trace), uc30 0.95, 32-256, 0.151968798',
 'parasail, uc30, full, 0.824957594',
 'parasail, uc30 0.95, full, 0.960366206']

In [14]:
data = csv_to_pandas(output)
data

,algorithm,dataset,size,time
0,ours (no trace),uc30,32-32,0.056956
1,ours (no trace),uc30 0.95,32-32,0.061715
2,ours (no trace),uc30,32-256,0.088623
3,ours (no trace),uc30 0.95,32-256,0.075700
4,ours (no trace),uc30,256-256,0.199376
5,ours (no trace),uc30 0.95,256-256,0.224719
6,ours (trace),uc30,32-256,0.169768
7,ours (trace),uc30 0.95,32-256,0.151969
8,parasail,uc30,full,0.824958
9,parasail,uc30 0.95,full,0.960366


Uniclust30 Benchmark (AVX2)

In [15]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("algorithm", axis = None),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = "algorithm"
).transform_filter(
    (datum.size == "32-256") | (datum.algorithm == "parasail")
)
t = c.mark_text(dy = -4, size = 8).encode(text = alt.Text("time", format = ".2f"), color = alt.value("black"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
).configure_range(
    category = {"scheme": "dark2"}
)
save(c, "uniclust30_bench.pdf")
c

alt.FacetChart(...)

Uniclust30 Block Size Benchmark (AVX2)

In [16]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", axis = None, sort = ["32-32", "32-256", "256-256"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = alt.Color("size", title = "block size", sort = ["32-32", "32-256", "256-256"])
).transform_filter(
    datum.algorithm == "ours (no trace)"
)
t = c.mark_text(dy = -4, size = 8).encode(text = alt.Text("time", format = ".2f"), color = alt.value("black"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
)
save(c, "uniclust30_size_bench.pdf")
c

alt.FacetChart(...)

## DNA Global Alignment Benchmark

In [17]:
output = !cd .. && cargo run --example nanopore_bench_global --release --features simd_avx2 --quiet
output

['# time (s)',
 'dataset, algorithm, time',
 'illumina, ours (1%-1%), 0.33111815499998787',
 'illumina, ours (1%-10%), 0.36067308799999676',
 'illumina, edlib, 1.078242317999999',
 'illumina, ksw_extz2_sse (1%), 0.7015643449999919',
 'illumina, ksw_extz2_sse (10%), 0.6619977030000476',
 'illumina, wfa2, 0.0395910699999757',
 'illumina, wfa2 adaptive, 0.033794843000004564',
 'illumina, parasail, 1.877281111000019',
 'nanopore 1kbp, ours (1%-1%), 0.35523873000000045',
 'nanopore 1kbp, ours (1%-10%), 0.4254908889999992',
 'nanopore 1kbp, edlib, 0.9824836550000021',
 'nanopore 1kbp, ksw_extz2_sse (1%), 0.6899555330000003',
 'nanopore 1kbp, ksw_extz2_sse (10%), 1.4918466799999923',
 'nanopore 1kbp, wfa2, 1.0744614449999976',
 'nanopore 1kbp, wfa2 adaptive, 0.9676685919999983',
 'nanopore 1kbp, parasail, 4.241416280000012',
 'nanopore <10kbp, ours (1%-1%), 1.2327692310000036',
 'nanopore <10kbp, ours (1%-10%), 1.745659388000004',
 'nanopore <10kbp, edlib, 4.181581389000004',
 'nanopore <10kb

In [18]:
data = csv_to_pandas(output)
data

,dataset,algorithm,time
0,illumina,ours (1%-1%),0.331118
1,illumina,ours (1%-10%),0.360673
2,illumina,edlib,1.078242
3,illumina,ksw_extz2_sse (1%),0.701564
4,illumina,ksw_extz2_sse (10%),0.661998
5,illumina,wfa2,0.039591
6,illumina,wfa2 adaptive,0.033795
7,illumina,parasail,1.877281
8,nanopore 1kbp,ours (1%-1%),0.355239
9,nanopore 1kbp,ours (1%-10%),0.425491


DNA Global Alignment Benchmark (AVX2)

In [19]:
algos = ["ours (1%-1%)", "ours (1%-10%)", "edlib", "ksw_extz2_sse (1%)", "ksw_extz2_sse (10%)", "wfa2", "wfa2 adaptive", "parasail"]
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("algorithm", sort = algos, title = None),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
    color = alt.Color("algorithm", legend = None)
)
t = c.mark_text(dy = -4, size = 8).encode(text = alt.Text("time", format = ".2f"), color = alt.value("black"))
c = (c + t).properties(
    width = 140,
    height = 140
).facet(
    facet = alt.Facet("dataset", title = None, header = alt.Header(orient = "top")),
    columns = 2
).resolve_scale(
    y = "independent"
).configure_axisY(
    labelPadding = 18,
    labelAlign = "left"
)
save(c, "dna_global_bench.pdf")
c

alt.FacetChart(...)

## Nanopore Data Benchmark Setup

To run the benchmarks below, you need to clone the following repos, place them in the same directory where this repo (block aligner) is located, and follow their setup instructions:
* [diff-bench-paper](https://github.com/Daniel-Liu-c0deb0t/diff-bench-paper)
* [adaptivebandbench](https://github.com/Daniel-Liu-c0deb0t/adaptivebandbench)

## Nanopore Data Benchmark

In [20]:
output = !cd .. && cargo run --example nanopore_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, time',
 'ours (no trace 32-32), nanopore 25kbp, 0.99076395',
 'ours (no trace 32-32), random, 2.39669256',
 'ours (trace 32-32), nanopore 25kbp, 1.378179032',
 'ours (trace 32-32), random, 3.38678551',
 'ours (trace 32-64), nanopore 25kbp, 1.6956111200000001',
 'ours (trace 32-64), random, 3.548437374']

In [21]:
data = csv_to_pandas(output)
data

,algorithm,dataset,time
0,ours (no trace 32-32),nanopore 25kbp,0.990764
1,ours (no trace 32-32),random,2.396693
2,ours (trace 32-32),nanopore 25kbp,1.378179
3,ours (trace 32-32),random,3.386786
4,ours (trace 32-64),nanopore 25kbp,1.695611
5,ours (trace 32-64),random,3.548437


In [22]:
output2 = !cd ../../diff-bench-paper/supplementary_data/benchmark_codes && ./custom_bench.sh

for i, o in enumerate(output2):
    if o.startswith("cells("):
        break
output2 = output2[i + 1:]

output2.insert(0, "algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail")
output2

['algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail',
 'editdist\t479857000\t171920000\t68242000\t720019000\t6880489\t0',
 'non-diff\t675709000\t271923000\t60448000\t1008080000\t27124786\t52',
 'diff-raw\t611150000\t205452000\t62678000\t879280000\t27291141\t32',
 'libgaba\t433122000\t150108000\t31507000\t614737000\t27121546\t53',
 'edlib\t28014226000\t19216508000\t105996000\t47336730000\t37\t0',
 'seqan\t90341741000\t0\t0\t90341741000\t0\t0']

In [23]:
data2 = csv_to_pandas(output2, d = "\t")
data2

,algorithm,fill time,trace time,convert time,total time,score,fail
0,editdist,479857000,171920000,68242000,720019000,6880489,0
1,non-diff,675709000,271923000,60448000,1008080000,27124786,52
2,diff-raw,611150000,205452000,62678000,879280000,27291141,32
3,libgaba,433122000,150108000,31507000,614737000,27121546,53
4,edlib,28014226000,19216508000,105996000,47336730000,37,0
5,seqan,90341741000,0,0,90341741000,0,0


In [24]:
cleaned2 = data2.drop(columns = ["trace time", "convert time", "total time", "score", "fail"])
cleaned2 = cleaned2.rename(columns = {"fill time": "time"})
cleaned2["time"] /= 1e9
cleaned2

,algorithm,time
0,editdist,0.479857
1,non-diff,0.675709
2,diff-raw,0.611150
3,libgaba,0.433122
4,edlib,28.014226
5,seqan,90.341741


In [25]:
cleaned = data.drop(index = [1, 3, 5])
cleaned = cleaned.drop(columns = ["dataset"])
cleaned = cleaned.append(cleaned2, ignore_index = True)
cleaned

,algorithm,time
0,ours (no trace 32-32),0.990764
1,ours (trace 32-32),1.378179
2,ours (trace 32-64),1.695611
3,editdist,0.479857
4,non-diff,0.675709
5,diff-raw,0.611150
6,libgaba,0.433122
7,edlib,28.014226
8,seqan,90.341741


25kbp Nanopore Reads Benchmark (AVX2)

In [26]:
chart1 = alt.Chart(cleaned).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm != "ours (trace 32-32)") & (datum.algorithm != "ours (no trace 32-32)") & (datum.algorithm != "ours (trace 32-64)"))

chart2 = alt.Chart(cleaned).mark_point(color = "red", filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm == "ours (trace 32-32)") | (datum.algorithm == "ours (no trace 32-32)") | (datum.algorithm == "ours (trace 32-64)"))

c = (chart1 + chart2).properties(
    width = 150,
    height = 150
)
save(c, "nanopore_bench.pdf")
c

alt.LayerChart(...)

## Sequence-to-Profile Alignment Benchmark

In [27]:
output = !cd .. && cargo run --example pssm_bench --release --features simd_avx2 --quiet
output

['size, time',
 '32-32, 0.149440358',
 '32-64, 0.175623995',
 '32-128, 0.205382135',
 '128-128, 0.206854299',
 'parasail, 0.596604504',
 '# Done!']

In [28]:
data = csv_to_pandas(output)
data

,size,time
0,32-32,0.149440
1,32-64,0.175624
2,32-128,0.205382
3,128-128,0.206854
4,parasail,0.596605


In [29]:
table = data.copy()
table = table.rename(columns = {"size": "block size"})
table["time"] = table["time"].map("{:.2}".format)
print(table)

  block size  time
0      32-32  0.15
1      32-64  0.18
2     32-128  0.21
3    128-128  0.21
4   parasail   0.6


SCOP Sequence-to-Profile Alignment Benchmark (AVX2)

In [30]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", title = "block size", sort = ["32-32", "32-64", "32-128", "128-128", "parasail"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
    color = alt.Color("size", sort = ["32-32", "32-64", "32-128", "128-128", "parasail"], legend = None)
).transform_filter(
    datum.size != "2048-2048"
).properties(
    width = 75,
    height = 100
)
t = c.mark_text(dy = -4, size = 8).encode(text = alt.Text("time", format = ".2f"), color = alt.value("black"))
c = c + t
save(c, "pssm_size_bench.pdf")
c

alt.LayerChart(...)

## WASM SIMD

[Wasmtime](https://wasmtime.dev/) is needed to run the webassembly code.

In [31]:
output = !CARGO_TARGET_WASM32_WASI_RUNNER="wasmtime --wasm-features simd --" cargo bench --target=wasm32-wasi --features simd_wasm --quiet -- --nocapture | grep 'bench:' | awk '{print $2"\t"$5}'
output

['warning: unused variable: `b`',
 '   --> src/simd128.rs:136:34',
 '    |',
 '136 | pub unsafe fn simd_step(a: Simd, b: Simd) -> Simd {',
 '    |                                  ^ help: if this is intentional, prefix it with an underscore: `_b`',
 '    |',
 '    = note: `#[warn(unused_variables)]` on by default',
 '',
 'bench_rustbio_aa_100_1000\t24,591,899',
 'bench_rustbio_aa_10_100\t260,176',
 'bench_scan_aa_1000_10000\t853,155',
 'bench_scan_aa_1000_10000_insert\t11,051,925',
 'bench_scan_aa_1000_10000_small\t597,115',
 'bench_scan_aa_1000_10000_trace\t1,963,569',
 'bench_scan_aa_100_1000\t77,247',
 'bench_scan_aa_100_1000_insert\t158,918',
 'bench_scan_aa_100_1000_small\t57,476',
 'bench_scan_aa_100_1000_trace\t217,913',
 'bench_scan_aa_10_100\t6,992',
 'bench_scan_aa_10_100_insert\t7,204',
 'bench_scan_aa_10_100_small\t5,438',
 'bench_scan_aa_10_100_trace\t84,637',
 'bench_scan_nuc_1000_10000\t561,650',
 'bench_scan_nuc_100_1000\t57,071']

In [32]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["rustbio_aa", "scan_aa", "scan_nuc"]
new_names = ["rust bio\tprotein", "ours\tprotein", "ours\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

ValueError: substring not found

In [33]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time


In [34]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (WASM SIMD)

In [35]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench_wasm.pdf")
c

/usr/local/anaconda3/lib/python3.8/site-packages/altair/utils/core.py:185: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  warnings.warn(
WARN FieldDef does not work with "log" scale. We are using "point" scale instead.


alt.Chart(...)